<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Predicting Shots Made Per Game by Kobe Bryant

_Authors: Kiefer Katovich (SF)_

---

In this lab you'll be using regularized regression penalties — ridge, lasso, and elastic net — to try and predict how many shots Kobe Bryant made per game during his career.

The Kobe Shots data set contains hundreds of columns representing different characteristics of each basketball game. Fitting an ordinary linear regression using every predictor would dramatically overfit the model, considering the limited number of observations (games) we have available. Plus, many of the predictors have significant multicollinearity. 


**Warning:** Some of these calculations are computationally expensive and may take a while to execute. It may be worthwhile to only use a portion of the data to perform these calculations, especially if you've experienced kernel issues in the past.

---

### 1) Load packages and data.

In [1]:
import numpy as np
import pandas as pd
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
kobe = pd.read_csv('./datasets/kobe_superwide_games.csv')

---

### 2) Examine the data.

- How many columns are there?
- Examine what the observations (rows) and columns represent.
- Why might regularization be particularly useful for modeling this data?

In [3]:
# A:
len(kobe.columns)

645

In [4]:
kobe.head(10)

,SHOTS_MADE,AWAY_GAME,SEASON_OPPONENT:atl:1996-97,SEASON_OPPONENT:atl:1997-98,SEASON_OPPONENT:atl:1999-00,SEASON_OPPONENT:atl:2000-01,SEASON_OPPONENT:atl:2001-02,SEASON_OPPONENT:atl:2002-03,SEASON_OPPONENT:atl:2003-04,SEASON_OPPONENT:atl:2004-05,...,ACTION_TYPE:tip_layup_shot,ACTION_TYPE:tip_shot,ACTION_TYPE:turnaround_bank_shot,ACTION_TYPE:turnaround_fadeaway_bank_jump_shot,ACTION_TYPE:turnaround_fadeaway_shot,ACTION_TYPE:turnaround_finger_roll_shot,ACTION_TYPE:turnaround_hook_shot,ACTION_TYPE:turnaround_jump_shot,SEASON_GAME_NUMBER,CAREER_GAME_NUMBER
0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
2,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4
4,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,5
5,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,6
6,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,7
7,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,8
8,4.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,9
9,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,10


#### Ans: Each row represents a basketball game while each column represents a characteristic/feature of the basketball game. Regularisation might be useful because the data contains many dummy columns. Hence, regularisation can perform some form of feature selection to reduce dummy columns that only provide noise.

---

### 3) Create predictor and target variables. Standardize the predictors.

Why is normalization necessary for regularized regressions?

Use the `sklearn.preprocessing` class `StandardScaler` to standardize the predictors.

In [6]:
# A: The betas of the predictors must be on the same scale. Only then can the algorithm determine which predictor is more or less important based on their numerical values.
from sklearn.preprocessing import StandardScaler
y = kobe['SHOTS_MADE']
X = kobe.drop('SHOTS_MADE', axis=1)
ss = StandardScaler()
X_transformed = ss.fit_transform(X)

---

### 4. Build a linear regression predicting `SHOTS_MADE` from the rest of the columns.

Cross-validate the $R^2$ of an ordinary linear regression model with 10 cross-validation folds.

How does it perform?

In [10]:
# A:
lr = LinearRegression()
lr_scores = cross_val_score(lr, X_transformed, y, cv=10)
lr_scores

array([-1.49205676e+29, -2.01405439e+28, -8.42919257e+27, -9.11670473e+27,
       -1.97317545e+28, -4.92473056e+27, -6.01157823e+27, -3.16384138e+27,
       -1.00482673e+28, -1.88859301e+28])

In [12]:
lr_scores.mean()

-2.496582194132251e+28

In [13]:
lr_scores.std()

4.184089903507845e+28

---

### 5) Find an optimal value for the ridge regression alpha using `RidgeCV`.

Go to the documentation and [read how RidgeCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).

> *Hint: Once the RidgeCV is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that ridge performs best when searching alphas through logarithmic space (`np.logspace`). This may take awhile to fit.


In [16]:
# A:
alphas = np.logspace(0, 10, 100)
ridge_best = RidgeCV(alphas=alphas, cv=10)
ridge_best.fit(X_transformed, y)
ridge_best.alpha_

849.7534359086438

---

### 6) Cross-validate the ridge regression $R^2$ with the optimal alpha.

Is it better than the linear regression? If so, why might this be?

In [17]:
# A:
ridge = Ridge(alpha=ridge_best.alpha_)
ridge_scores = cross_val_score(ridge, X_transformed, y, cv=10)
ridge_scores

array([0.57260259, 0.52815995, 0.48678712, 0.59767973, 0.52611268,
       0.52928677, 0.52809688, 0.41219229, 0.40205367, 0.44615846])

In [18]:
ridge_scores.mean()

0.5029130142501572

In [19]:
ridge_scores.std()

0.06182781030096776

#### Ridge scores are much better. This is because we are able to optimise for the most optimal value of alpha when using ridge. Ridge also penalises coefficients of predictors that are not deemed to be useful

---

### 7) Find an optimal value for lasso regression alpha using `LassoCV`.

Go to the documentation and [read how LassoCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html). It is very similar to `RidgeCV`.

> *Hint: Again, once the `LassoCV` is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that lasso, unlike ridge, performs best when searching for alpha through linear space (`np.linspace`). However, you can actually let the LassoCV decide what alphas to use itself by setting the keyword argument `n_alphas=` to however many alphas you want it to search over. We recommend letting scikit-learn choose the range of alphas.

_**Tip:** If you find your CV taking a long time and you're not sure if it's working, set `verbose =1`._

In [21]:
# A:
lasso_best = LassoCV(n_alphas=400, cv=10, verbose=1)
lasso_best.fit(X_transformed, y)
lasso_best.alpha_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

0.06801622021798569

---

### 8) Cross-validate the lasso $R^2$ with the optimal alpha.

Is it better than the linear regression? Is it better than ridge? What do the differences in results imply about the issues with the data set?

In [22]:
# A:
lasso = Lasso(alpha=lasso_best.alpha_)
lasso_scores = cross_val_score(lasso, X_transformed, y, cv=10)
lasso_scores

array([0.61595069, 0.52641331, 0.53889668, 0.61318714, 0.54630234,
       0.55541629, 0.52712688, 0.44850472, 0.46462769, 0.51845755])

In [23]:
lasso_scores.mean()

0.535488327703056

In [24]:
lasso_scores.std()

0.051055533530402555

#### Lasso performs better than the ridge. This could be because ridge penalises the coefficients of irrelevant features but lasso removes them completely.

---

### 9) Look at the coefficients for variables in the lasso.

1. Show the coefficient for variables, ordered from largest to smallest coefficient by absolute value.
2. What percent of the variables in the original data set are "zeroed-out" by the lasso?
3. What are the most important predictors for how many shots Kobe made in a game?

> **Note:** If you only fit the lasso within `cross_val_score`, you'll have to refit it outside of that function to pull out the coefficients.

In [32]:
# A:
lasso.fit(X_transformed, y)
# Create df to store predictor name, coef and their absolute values
lasso_coefs_df = pd.DataFrame({'feature':X.columns, 'coef': lasso.coef_, 'absp_coef': np.abs(lasso.coef_)})
# Sort by absolute coefficientsin descending order
lasso_coefs_df.sort_values('absp_coef', inplace=True, ascending=False)
lasso_coefs_df.head(10)

,feature,coef,absp_coef
579,COMBINED_SHOT_TYPE:jump_shot,1.233343,1.233343
574,SHOT_TYPE:2pt_field_goal,0.866846,0.866846
566,SHOT_ZONE_BASIC:restricted_area,0.452825,0.452825
577,COMBINED_SHOT_TYPE:dunk,0.274886,0.274886
611,ACTION_TYPE:jump_shot,-0.272023,0.272023
575,SHOT_TYPE:3pt_field_goal,0.140657,0.140657
582,SECONDS_REMAINING,0.132441,0.132441
612,ACTION_TYPE:layup_shot,-0.129564,0.129564
423,SEASON_OPPONENT:sac:1999-00,0.126929,0.126929
561,SHOT_ZONE_BASIC:above_the_break_3,0.118412,0.118412


In [35]:
len(lasso_coefs_df[lasso_coefs_df['coef'] == 0])

508

#### 508 features are reduced to 0. Important predictors are those with the higehst coefficients.

---

### 10) Find an optimal value for elastic net regression alpha using `ElasticNetCV`.

Go to the documentation and [read how ElasticNetCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html).

Note that here you'll be optimizing both the alpha parameter and the l1_ratio:
- `alpha`: Strength of regularization.
- `l1_ratio`: Amount of ridge vs. lasso (0 = all ridge, 1 = all lasso).
    
Do not include 0 in the search for `l1_ratio` — it won't allow it and will break.

You can use `n_alphas` for the alpha parameters instead of setting your own values, which we highly recommend.

Also, be careful setting too many l1_ratios over cross-validation folds in your search. It can take a long time if you choose too many combinations and, for the most part, there are diminishing returns in this data.

In [37]:
# A:
# Get 20 numbers between 0.01 and 1
l1_ratio_list = np.linspace(0.01, 1.0, 20)
elastic_best = ElasticNetCV(l1_ratio=l1_ratio_list, n_alphas=100, cv=10, verbose=1)
elastic_best.fit(X_transformed, y)
elastic_best.alpha_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

0.07354632415031567

In [38]:
elastic_best.l1_ratio_

0.9478947368421053

---

### 11) Cross-validate the elastic net $R^2$ with the optimal alpha and l1_ratio.

How does it compare to the ridge and lasso regularized regressions?

In [39]:
# A:
elastic = ElasticNet(alpha=elastic_best.alpha_, l1_ratio=elastic_best.l1_ratio_)
elastic_scores = cross_val_score(elastic, X_transformed, y, cv=10)
elastic_scores.mean()

0.5358127053101619

#### Performance is similar to lasso. This is because the l1_ratio selected is 0.94, which is very close to 1. 1 = all lasso.

---

### 12. [Bonus] Compare the residuals for ridge and lasso visually.


In [ ]:
# A: Maybe a jointplot?